# Generating the Solutions for the Selected Problems
Here, I want to use **Qwen2.5-Coder-14B-Instruct** to generate the solutions for the selected problems. Each of them will be written in Python, C#, and Typescript. The model is available on [Hugging Face](https://huggingface.co/Qwen/Qwen2.5-Coder-14B-Instruct).

In [69]:
# !pip install transformers

In [70]:
import re
import os
import pandas as pd
from tqdm import tqdm
from enum import Enum
from transformers import AutoTokenizer, AutoModelForCausalLM

## Selected Problems Loading
Firstly, I need to load the selected problems dataframe.

In [71]:
csv_file = '/datasets/leetcode_selected_problems.csv'
csv_full_path = os.getcwd()+csv_file
if os.path.exists(csv_full_path):
    selected_problems = pd.read_csv(csv_full_path)
else:
    selected_problems = None
    print("File not found.")

In [72]:
selected_problems["solution"] = ''

In [73]:
selected_problems.head()

,title,description,difficulty,solution
0,Relative Ranks,You are given an integer array `score` of size...,Easy,
1,Truncate Sentence,A sentence is a list of words that are separat...,Easy,
2,Perform String Shifts,You are given a string `s` containing lowercas...,Easy,
3,Invert Binary Tree,"Given the `root` of a binary tree, invert the ...",Easy,
4,Merge Strings Alternately,You are given two strings `word1` and `word2`....,Easy,


## Loading Qwen2.5-Coder-14B-Instruct Model

In [74]:
class Language(Enum):
    PYTHON = "python"
    TYPESCRIPT = "typescript"
    C_SHARP = "csharp"

In [75]:
model_name = "Qwen/Qwen2.5-Coder-14B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [76]:
c_sharp_prompt = "You are a C# developer. Please, solve this problem avoid using any built-in functions or libraries that simplify algorithmic steps:\n"
python_prompt = "You are a Python developer. Please, solve this problem avoid using any built-in functions or libraries that simplify algorithmic steps:\n"
typescript_prompt = "You are a Typescript developer. Please, solve this problem avoid using any built-in functions or libraries that simplify algorithmic steps:\n"

In [77]:
def get_code_from_text(text, programming_language):
    code_match = re.search(r'```(.*?)```', text, re.DOTALL)
    if code_match:
        extracted_code = code_match.group(1).strip().replace(programming_language.value+'\n', '')
        return extracted_code
    else:
        return "No code found in the generated output"

In [78]:
max_context_length = model.config.max_position_embeddings

def get_generated_code(prompt, programming_language):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = inputs['input_ids']
    context_length = input_ids.shape[1]
    if context_length >= max_context_length:
        print(f"The prompt is too long: {context_length}; Maximum: {max_context_length}")
        return None

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_context_length-1)
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return get_code_from_text(generated_answer, programming_language)

In [79]:
generated_code = get_generated_code(c_sharp_prompt+selected_problems.iloc[0]["description"], Language.C_SHARP)
print(generated_code)

using System;

public class Solution
{
    public string[] FindRelativeRanks(int[] score)
    {
        int n = score.Length;
        int[] sortedScores = new int[n];
        
        // Copy the original scores to a new array
        for (int i = 0; i < n; i++)
        {
            sortedScores[i] = score[i];
        }
        
        // Sort the copied scores in descending order
        for (int i = 0; i < n - 1; i++)
        {
            for (int j = i + 1; j < n; j++)
            {
                if (sortedScores[i] < sortedScores[j])
                {
                    // Swap
                    int temp = sortedScores[i];
                    sortedScores[i] = sortedScores[j];
                    sortedScores[j] = temp;
                }
            }
        }
        
        // Create a dictionary to map scores to ranks
        string[] ranks = new string[n];
        for (int i = 0; i < n; i++)
        {
            if (i == 0)
            {
                ranks[sortedS

## Code Generation

In [80]:
def generate_solution(start_prompt, programming_language):
    problems_with_solutions = selected_problems
    for index, row in tqdm(problems_with_solutions.iterrows()):
         problems_with_solutions.loc[index, "solution"] = get_generated_code(start_prompt+row["description"], programming_language)
    
    return problems_with_solutions

In [58]:
c_sharp_solutions = generate_solution(c_sharp_prompt, Language.C_SHARP)
c_sharp_solutions.to_csv('datasets/leetcode_selected_problems_c_sharp_14B.csv', index=False)

30it [3:33:19, 426.65s/it]


In [81]:
typescript_solutions = generate_solution(typescript_prompt, Language.TYPESCRIPT)
typescript_solutions.to_csv('datasets/leetcode_selected_problems_typescript_14B.csv', index=False)

30it [3:19:34, 399.17s/it]


In [82]:
python_solutions = generate_solution(python_prompt, Language.PYTHON)
python_solutions.to_csv('datasets/leetcode_selected_problems_python_14B.csv', index=False)

30it [3:05:37, 371.25s/it]
